### Installing packages

In [2]:
# we use natsort package to sort those missing leading zero files 
!pip install natsort
!pip install pandas

  Using cached natsort-8.4.0-py3-none-any.whl.metadata (21 kB)
Using cached natsort-8.4.0-py3-none-any.whl (38 kB)


### Defining ANSI codes for colored text prints 

In [3]:
# ANSI escape codes, to make log prints nicer
RED = "\033[31m"
GREEN = "\033[32m"
YELLOW = "\033[33m"
BLUE = "\033[34m"
BOLD = "\033[1m"
ITALIC = "\x1B[3m"
UNDERLINED = "\033[4m"
RESET = "\033[0m"
WHITE_BG    = "\x1b[47m\033[30m" # adding \033[30m makes text black
GREEN_BG    = "\x1b[102m\033[30m" # adding \033[30m makes text black

# https://jakob-bagterp.github.io/colorist-for-python/ansi-escape-codes/standard-16-colors/#bright-colors_2

### Imports

In [27]:
import pandas as pd
import os
from natsort import os_sorted
from datetime import datetime, timedelta
import requests
import zipfile
import warnings 
import urllib3

from dotenv import dotenv_values
from sqlalchemy import create_engine, types, text

# we'll suppress the "missing SSL certificate" warnings while downloading files
warnings.simplefilter("ignore", urllib3.exceptions.InsecureRequestWarning) 

## Data Download
**Sources**  
>Raw Data: https://transtats.bts.gov/PREZIP/  
>Website: https://transtats.bts.gov

#### working scenario: 
1. choose a time period for your flights data<br>**NOTE:** usually latest month available is = now - 3 months
2. in the first cell: 
    - update `start` for the start date
    - update `length` for the number of month 
3. execute all other cells in this notebook
   <br>**NOTE:** the steps are optimized for multiple months period, but would also work for 1 month  
  
<details>
<summary style="color:grey">all steps explained</summary>

1. decide on the period and update `start` and `length` variables
2. if not yet created, add 2 folders inside `\da-analytics-engineering-project\` repo:
     - `downloads`
     - and `downloads/extracted`
3. choose the time period for the flights data (starting month, total number of months)    
4. under the [transtats URL](https://transtats.bts.gov/PREZIP/) above find files names starting with  
`"On_Time_Reporting_Carrier_On_Time_Performance_1987_present_####_##.zip"`  
- each ZIP file contains a CSV file for **one month** of data (indicated as ####_##)  
- download desired zipfiles to the `downloads` folder  
5. extract the CSV files into the `downloads/extracted` folder
</details>

In [5]:
# 1. Decide on starting month and total number of months
start = '08.2017' # Enter the starting month and the year (MM.YYYY)
length = 2 # How many months do you need?

In [6]:
# 2. Create folders for the zip files download and for the CSV-files extraction
os.makedirs('./downloads/extracted', exist_ok=True)

In [7]:
# 3. Create a list of months for the flight

# Generate list of MM.YYYY values for one year
def generate_year_list(start, length):
    start_date = datetime.strptime(start, '%m.%Y')
    return [f"{dt.year}_{dt.month}" for dt in
        (start_date + timedelta(days=31 * i) for i in range(length))]

# MM_YYYY values for the period lenght
year_month_list = generate_year_list(start, length)

print(year_month_list)


['2017_8', '2017_9']


In [8]:
# 4. Download ZIP files (~35 seconds per one file)

# Define the URL of the ZIP file
base_url = 'https://transtats.bts.gov/PREZIP/'
download_time = timedelta(0) # for time logging
disk_space_zip = 0

for year_month in year_month_list:

    # Define the URL of the ZIP file and the CSV file
    zip_name = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year_month}.zip'

    print(f'\n ⏳ This should take {RED}~35 seconds...{RESET}\n\n    ⬇️ {BLUE}downloading:{RESET} {zip_name}')
    print(f'    🐌 {YELLOW}wait for it...{RESET}', end='\r')
    start_time = datetime.now()

    # Send a HTTP request to the specified URL and save the response content
    response = requests.get(base_url+zip_name, verify=False) # we ignore the SSL certificate warnings

    with open(f'./downloads/{zip_name}', 'wb') as file: # save the ZIP in "downloads folder"
        file.write(response.content)
        print(f'    ✅ {GREEN}file saved:{RESET} {zip_name}', end=' ')
    
    # assessing the size of the downloaded file
    file_size = os.path.getsize(f'./downloads/{zip_name}') 
    size_in_mb = file_size / (1024 ** 2) 
    print(f'{GREEN}({size_in_mb:.2f} MB){RESET}\n')
    disk_space_zip += file_size

    # just some fun with basic time logging  
    end_time = datetime.now()
    time_difference = end_time - start_time
    download_time = download_time + time_difference
    if (time_difference.seconds // 60) < 1:
        print(f' 🦊 Actually it took: {YELLOW}{time_difference.seconds % 60} seconds\n{RESET}','-'*80)
    else:
        print(f' 🦊 Actually it took: {YELLOW}{time_difference.seconds // 60} minutes and {time_difference.seconds % 60} seconds\n{RESET}','-'*80)
print(f' 🦊 Total Download Time: {YELLOW}{download_time.seconds // 60} minutes and {download_time.seconds % 60} seconds\n{RESET}')
print(f' 🐹 Used Disk Space: {GREEN}({(disk_space_zip / (1024 ** 2)):.2f} MB){RESET}')


 ⏳ This should take ~35 seconds...

    ⬇️ downloading: On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2017_8.zip
    ✅ file saved: On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2017_8.zip (26.06 MB)

 🦊 Actually it took: 49 seconds
 --------------------------------------------------------------------------------

 ⏳ This should take ~35 seconds...

    ⬇️ downloading: On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2017_9.zip
    ✅ file saved: On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2017_9.zip (22.70 MB)

 🦊 Actually it took: 43 seconds
 --------------------------------------------------------------------------------
 🦊 Total Download Time: 1 minutes and 32 seconds

 🐹 Used Disk Space: (48.76 MB)


In [9]:
# 5. Extracting CSV files only

disk_space_csv = 0

for year_month in year_month_list:

    # Define the name of the ZIP file and the CSV file
    zip_name = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year_month}.zip'
    csv_name = f'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_{year_month}.csv'

    # Open the downloaded ZIP file
    with zipfile.ZipFile(f'./downloads/{zip_name}', 'r') as zip_ref:
        # Extract the CSV file
        zip_ref.extract(csv_name, path='./downloads/extracted/') # save the CSV in "downloads folder"
        print(f'    🍌 extracted "{csv_name}', end=' ')
        
    # assessing the size of the extracted file
    file_size = os.path.getsize(f'./downloads/extracted/{csv_name}') 
    size_in_mb = file_size / (1024 ** 2) 
    print(f"{GREEN}({size_in_mb:.2f} MB){RESET}\n")
    disk_space_csv += file_size

print('-'*80,f'\n 🐹 Used Disk Space: {GREEN}({(disk_space_csv / (1024 ** 2)):.2f} MB){RESET}')


    🍌 extracted "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2017_8.csv (219.43 MB)

    🍌 extracted "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2017_9.csv (196.12 MB)

-------------------------------------------------------------------------------- 
 🐹 Used Disk Space: (415.55 MB)


# Data Wrangling

### 1. adding all CSV file names to a list

In [10]:
# Add all file names from the "extracted" folder to a list
file_names = os.listdir('./downloads/extracted/')

# make sure only the data files are in the list
file_names_unordered = [fname for fname in file_names if fname.startswith("On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_")]

# using os_sorted function (from natsort) - able to sort strings with numbers ['2','1','11']
# sorted(['2','1','11']) # for comparison
data_files = os_sorted(file_names_unordered)

data_files

['On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2017_8.csv',
 'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2017_9.csv']

In [11]:
# we have 110 columns in each CSV...
file_check = pd.read_csv(f'./downloads/extracted/{data_files[0]}', low_memory = False)
file_check.shape

(510451, 110)

In [12]:
# original column names are not optimal and need renaming...
print(file_check.columns.to_list())

['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate', 'Reporting_Airline', 'DOT_ID_Reporting_Airline', 'IATA_CODE_Reporting_Airline', 'Tail_Number', 'Flight_Number_Reporting_Airline', 'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'Origin', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'Dest', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'CRSDepTime', 'DepTime', 'DepDelay', 'DepDelayMinutes', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk', 'Cancelled', 'CancellationCode', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Flights', 'Distance', 'DistanceGroup', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'FirstDepTime', 'TotalAddGTime'

### 2. Defining functions
<font size=4>
<ul><li>column filter<li>renaming columns<li>changing data types</ul>
</font>

In [14]:
# select columns to keep
def cols_to_keep(flights_raw):
    columns_to_keep = [
        "FlightDate",
        "DepTime",
        "CRSDepTime",
        "DepDelay",
        "ArrTime",
        "CRSArrTime",
        "ArrDelay",
        "Reporting_Airline",
        "Tail_Number",
        "Flight_Number_Reporting_Airline",
        "Origin",
        "Dest",
        "AirTime",
        "ActualElapsedTime",
        "Distance",
        "Cancelled",
        "Diverted",
    ]
    flights = flights_raw.loc[:, columns_to_keep]
    return flights

In [15]:
# rename columns
def rename_cols(flights):
    new_column_names = {
        'FlightDate': 'flight_date',
        'DepTime': 'dep_time',
        'CRSDepTime': 'sched_dep_time',
        'DepDelay': 'dep_delay',
        'ArrTime': 'arr_time',
        'CRSArrTime': 'sched_arr_time',
        'ArrDelay': 'arr_delay',
        'Reporting_Airline': 'airline',
        'Tail_Number': 'tail_number',
        'Flight_Number_Reporting_Airline': 'flight_number',
        'Origin': 'origin',
        'Dest': 'dest',
        'AirTime': 'air_time',
        'ActualElapsedTime': 'actual_elapsed_time',
        'Distance': 'distance',
        'Cancelled': 'cancelled',
        'Diverted': 'diverted'
    }
    flights.rename(columns=new_column_names, inplace=True)
    return flights

In [16]:
# change datatype
def change_dtypes(flights):
    types_change = {
        'flight_date': 'datetime64[ns]',
        'dep_time': 'Int16',
        'sched_dep_time': 'Int16',
        'dep_delay': 'Int16',
        'arr_time': 'Int16',
        'sched_arr_time': 'Int16',
        'arr_delay': 'Int16',
        'airline': 'O',
        'tail_number': 'O',
        'flight_number': 'Int16',
        'origin': 'O',
        'dest': 'O',
        'air_time': 'Int16',
        'actual_elapsed_time': 'Int16',
        'distance': 'Int16',
        'cancelled': 'Int16',
        'diverted': 'Int16'
    }
    flights = flights.astype(types_change)
    return flights

In [17]:
data_files

['On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2017_8.csv',
 'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2017_9.csv']

In [18]:
# list for separate dataframes
flights_list = []

#  loop over the extracted csv files and execute functions 
for file in data_files:
    print(file)
    print('reading...', end=" ")
    flights_raw = pd.read_csv(f'./downloads/extracted/{file}', low_memory = False) # read as a dataframe
    
    flights_select = cols_to_keep(flights_raw) # select columns to keep
    print('filter colums...', end=" ")
    flights_rename = rename_cols(flights_select) # rename columns
    print('rename colums...', end=" ")
    flights_dtypes = change_dtypes(flights_rename) # change data types
    print('change dtypes...', end=" ")
    
    flights_list.append(flights_dtypes) # add to the list of dateframes
    print(f'✅ {GREEN}appended to flight_list{RESET}\n')
    
print(f'Done. The list has {len(flights_list)} elements')

On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2017_8.csv
reading... filter colums... rename colums... change dtypes... ✅ appended to flight_list

On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2017_9.csv
reading... filter colums... rename colums... change dtypes... ✅ appended to flight_list

Done. The list has 2 elements


In [20]:
# concatenate the list of dataframes to a one dataframe
flights_all = pd.concat(flights_list)

In [21]:
# sort dataframe 
flights_all.sort_values(['flight_date','sched_dep_time'], inplace=True)

In [22]:
# countercheck the time period
flights_all['flight_date'].min(), flights_all['flight_date'].max()

(Timestamp('2017-08-01 00:00:00'), Timestamp('2017-09-30 00:00:00'))

In [23]:
flights_all

,flight_date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,airline,tail_number,flight_number,origin,dest,air_time,actual_elapsed_time,distance,cancelled,diverted
275107,2017-08-01,2353,1,-8,520,546,-26,UA,N68807,1197,SFO,IAH,190,207,1635,0,0
275534,2017-08-01,1,5,-4,822,824,-2,UA,N505UA,1796,SFO,EWR,288,321,2565,0,0
399884,2017-08-01,2,5,-3,451,502,-11,NK,N622NK,298,LAS,IAH,147,169,1222,0,0
164532,2017-08-01,9,10,-1,800,807,-7,AA,N171AA,1905,SFO,CLT,256,291,2296,0,0
275865,2017-08-01,9,10,-1,615,609,6,UA,N77865,2382,SFO,ORD,215,246,1846,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347940,2017-09-30,<NA>,2359,<NA>,<NA>,348,<NA>,B6,N658JB,839,JFK,BQN,<NA>,<NA>,1576,1,0
348103,2017-09-30,11,2359,12,753,757,-4,B6,N971JB,1248,LAS,JFK,255,282,2248,0,0
348169,2017-09-30,<NA>,2359,<NA>,<NA>,347,<NA>,B6,N966JB,1503,JFK,SJU,<NA>,<NA>,1598,1,0
423072,2017-09-30,2349,2359,-10,531,609,-38,UA,N68805,214,SFO,ORD,205,222,1846,0,0


# Saving the combined dataset<p><font size=5>(just as a backup)</font>

In [24]:
# define the file name for the combined CSV file (using period's first and last month)
output_file_name = f'flights_from_{year_month_list[0]}_until_{year_month_list[-1]}.csv'
output_file_name

'flights_from_2017_8_until_2017_9.csv'

In [25]:
# create folder 'data'
os.makedirs('./data', exist_ok=True)

In [ ]:
# saving
flights_all.to_csv(f'./data/{output_file_name}', index=False)

print(f' ✅ {GREEN}Combined Dataset Saved:{RESET} {output_file_name}', end=' ')

# assessing the size of the extracted file
file_size = os.path.getsize(f'./data/{output_file_name}') 
size_in_mb = file_size / (1024 ** 2) 
print(f"{GREEN}({size_in_mb:.2f} MB){RESET}\n")

# Get the absolute path
absolute_path = os.path.abspath(f'./data/{output_file_name}')
print(absolute_path)



 ✅ Combined Dataset Saved: flights_from_2017_8_until_2017_9.csv (68.12 MB)

/Users/mvolman/Documents/da-bash-tutorial-hh-analytics-24-4-main/extra work - 140225/data/flights_from_2017_8_until_2017_9.csv


### We got the data in a dataframe. Now it needs to be loaded into our DB.

_________

In [ ]:
#

In [28]:
# defining data types for the DB
flights_dtypes = {
    'flight_date': types.DateTime,
    'dep_time': types.Integer,
    'sched_dep_time': types.Integer,
    'dep_delay': types.Integer,
    'arr_time': types.Integer,
    'sched_arr_time': types.Integer,
    'arr_delay': types.Integer,
    'airline': types.String,
    'tail_number': types.String,
    'flight_number': types.Integer,
    'origin': types.String,
    'dest': types.String,
    'air_time': types.Integer,
    'actual_elapsed_time': types.Integer,
    'distance': types.Integer,
    'cancelled': types.Integer,
    'diverted': types.Integer
}

In [30]:
%whos DataFrame

Variable         Type         Data/Info
---------------------------------------
file_check       DataFrame            Year  Quarter  Mo<...>10451 rows x 110 columns]
flights_all      DataFrame           flight_date  dep_t<...>969178 rows x 17 columns]
flights_raw      DataFrame            Year  Quarter  Mo<...>58727 rows x 110 columns]
flights_rename   DataFrame           flight_date  dep_t<...>458727 rows x 17 columns]
flights_select   DataFrame           flight_date  dep_t<...>458727 rows x 17 columns]
